In [0]:
#Librerias que se usaran
import pandas as pd  # Libreria para administrar tablas, y realizar trabajos con distintas formas de tablas o dataframes
import numpy as np   # Libreria para poder hacer operaciones matematicas y matriciales
import matplotlib.pyplot as plt # Libreria para realizar graficos 
from tabulate import tabulate   # Permite formatear y mostrar de mejor manera los datos tabulares
import seaborn as sns  # Libreria para realizar graficos y vizualizaciones
import psycopg2        # Libreria que permite la conexion con PostgresSQL
from matplotlib.backends.backend_pdf import PdfPages # Libreria que permite exportar graficos en pdf

#Configuramos pandas para que podamos vizualizar todas las columnas y filas la estadistica descriptiva de todas las variables
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
#Configuramos pandas para que lanze valores con una precision de hasta 6 decimales
pd.set_option('display.float_format', '{:.6f}'.format)

## 1. Extraemos los datos del C4M

In [0]:
# Información de la conexión a PostgreSQL
host = "srv-postgres-d4m.postgres.database.azure.com"
database = "ControlSenseDB"
user = "administrador"
password = "Protobuffers2024"

# Establecer la conexión a la base de datos
try:
    connection = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )

    # Crear un cursor para ejecutar comandos SQL
    cursor = connection.cursor()

    # Establecer la zona horaria antes de ejecutar la consulta
    time_zone_query = "SET TIME ZONE 'America/Lima';"
    cursor.execute(time_zone_query)

    # Tu consulta SQL equipos de acarreo a considerar en c4mequipos y datacamion: 61,63,64,65,68,69,71,74,103,122,125,126,198,199,201,204,205
    # equipos acarreo disponibles en telemetria h4m(2dias) : 61,65,68,126,199,201,*204*,205
    tu_query_sql = '''
    select main.*, e.id_equipo, e.nombre from (SELECT *
    FROM public.getsensorsvaluesmod(201, '2024-02-27 07:00:30.612-05', '2024-03-03 06:27:30.612-05') a --data_camion
    left join tp_tramosstat b
    on b.id = (select id from tp_tramosstat where id_tptramosstat = a.tramosidsnew_t and tiem_creac < a.instant_date_t
                    order by tiem_creac desc limit 1)) main
    left join ts_equipos e
    on main.eq_id = e.id_equipo				
    where main.nombre_tramo is not null
    order by main.instant_date_t asc
    '''
    # Ejecutar la consulta
    cursor.execute(tu_query_sql)

    # Obtener los resultados en un DataFrame de pandas
    resultados17 = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

    # Cerrar el cursor y la conexión
    cursor.close()
    connection.close()

    # Hacer lo que necesites con los resultados
    print(resultados17.head())

except psycopg2.Error as e:
    print("Error al conectar a la base de datos PostgreSQL:", e)

In [0]:
resultados17.head()

In [0]:
resultados17.shape

#### Guardarmos directamente el dfpandas en un csv en el BlobStorage
Nota importante: Debes comentar la variable "connection_string" Si NO deja hacer COMMIT DEL #CODIGO EN GIT,GITHUB

In [0]:
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Convertir el DataFrame de Pandas a CSV
#csv_data = resultados15.to_csv("datos_raw_shougang_equipo199.csv", index=False)
csv_data = resultados17.to_csv(index=False)

# Obtener conection,de Azure DataLake,interfaz de Azure (Claves de acceso: Key1) (Debes comentar esta variable Si NO deja hacer COMMIT DEL 
# CODIGO EN GIT,GITHUB)
#connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakemlopsd4m;AccountKey=iWT8t74/#XlqcqoR03keDVtFZPzr0PB9zDffMPaLWMUBIAjUww8uYAVkc9xRkcBtvTmUHKBvd1sB3+ASt6mGgcQ==;EndpointSuffix=core.windows.net'

# Conectar al Blob Storage de Azure
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Nombre del contenedor y archivo en el Blob Storage
container_name = "raw/proyectocongestion_raw/fuentedatos_c4m/operacion_shougang/"
blob_name = "datos_raw_shougang_equipo201.csv"

# Subir el archivo CSV al Blob Storage
# with open("datos_raw_shougang_equipo61.csv", "rb") as data:
#     blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
#     blob_client.upload_blob(data)

# Subir el archivo CSV al Blob Storage
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
blob_client.upload_blob(csv_data)

### Unir todos los csv(datalake-equiposacarreo), Se leen varios dfs(csv) en un solo df
Nota importante: Debes comentar la variable "conection_string" Si NO deja hacer COMMIT DEL #CODIGO EN GIT,GITHUB

In [0]:
import pandas as pd
from azure.storage.blob import BlobServiceClient

# Obtener conection,de Azure DataLake,interfaz de Azure (Claves de acceso: Key1) (Debes comentar esta variable Si NO deja hacer COMMIT DEL #CODIGO EN GIT,GITHUB)
# connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakemlopsd4m;AccountKey=iWT8t74/#XlqcqoR03keDVtFZPzr0PB9zDffMPaLWMUBIAjUww8uYAVkc9xRkcBtvTmUHKBvd1sB3+ASt6mGgcQ==;EndpointSuffix=core.windows.net'

# Crear el cliente del servicio Blob
# blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Nombre del contenedor en el Blob Storage
container_name = "raw/proyectocongestion_raw/fuentedatos_c4m/operacion_shougang/"

# Lista para almacenar los DataFrames de cada archivo CSV
dfs = []

# Nombres de los archivos CSV
csv_files = ["datos_raw_shougang_equipo61.csv","datos_raw_shougang_equipo65.csv","datos_raw_shougang_equipo68.csv","datos_raw_shougang_equipo126.csv","datos_raw_shougang_equipo199.csv","datos_raw_shougang_equipo199.csv",
             "datos_raw_shougang_equipo201.csv","datos_raw_shougang_equipo205.csv"]

# Leer cada archivo CSV y guardarlos en DataFrames individuales
for csv_file in csv_files:
    # Descargar el archivo CSV del Blob Storage
    with open(csv_file, "wb") as my_blob:
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=csv_file)
        blob_data = blob_client.download_blob()
        blob_data.readinto(my_blob)
    
    # Leer el archivo CSV en un DataFrame de Pandas
    df = pd.read_csv(csv_file)
    
    # Agregar el DataFrame a la lista
    dfs.append(df)

# Concatenar todos los DataFrames en uno solo al final
result_df = pd.concat(dfs, ignore_index=True)

# Visualizar el DataFrame resultante
result_df.head()

#### Guardar el csv consolidado C4M (se guarda 1 solo archivo de csv)
Nota importante: Debes comentar la variable "conection_string" Si NO deja hacer COMMIT DEL #CODIGO EN GIT,GITHUB

In [0]:
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Convertir el DataFrame de Pandas a CSV
result_df.to_csv("datos_raw_shougang_c4m.csv", index=False)

# Obtener conection,de Azure DataLake,interfaz de Azure (Claves de acceso: Key1) (Debes comentar esta variable Si NO deja hacer COMMIT DEL #CODIGO EN GIT,GITHUB)
# connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakemlopsd4m;AccountKey=iWT8t74/#XlqcqoR03keDVtFZPzr0PB9zDffMPaLWMUBIAjUww8uYAVkc9xRkcBtvTmUHKBvd1sB3+ASt6mGgcQ==;EndpointSuffix=core.windows.net'

# Conectar al Blob Storage de Azure
blob_service_client = BlobServiceClient.from_connection_string(conection_string)

# Nombre del contenedor y archivo en el Blob Storage
container_name = "raw/proyectocongestion_raw/fuentedatos_c4m/operacion_shougang/"
blob_name = "datos_raw_shougang_c4m.csv"

# Subir el archivo CSV al Blob Storage
with open("datos_raw_shougang_c4m.csv", "rb") as data:
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data)

#### Leer csv consolidado c4m (se lee solo 1 CSV)
Nota importante: Debes comentar la variable "conection_string" Si NO deja hacer COMMIT DEL #CODIGO EN GIT,GITHUB

In [0]:
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Obtener la conexión al Blob Storage de Azure
#connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakemlopsd4m;AccountKey=iWT8t74/#XlqcqoR03keDVtFZPzr0PB9zDffMPaLWMUBIAjUww8uYAVkc9xRkcBtvTmUHKBvd1sB3+ASt6mGgcQ==;EndpointSuffix=core.windows.net'

# Crear el cliente del servicio Blob
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Nombre del contenedor y archivo en el Blob Storage
container_name = "raw/proyectocongestion_raw/fuentedatos_c4m/operacion_shougang/"
blob_name = "datos_raw_shougang_c4m.csv"

# Descargar el archivo CSV del Blob Storage
with open("datos_raw_shougang_c4m.csv", "wb") as my_blob:
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_data = blob_client.download_blob()
    blob_data.readinto(my_blob)

# Leer el archivo CSV en un DataFrame de Pandas
datos = pd.read_csv("datos_raw_shougang_c4m.csv")

# Visualizar el DataFrame
datos.head()

In [0]:
resultados_equipos.head()

In [0]:
# Realizar left join
merged_df = pd.merge(datos, resultados_equipos[['id_equipo','nombre']], left_on='eq_id', right_on='id_equipo', how='left')
merged_df.head()

### Extraer datos de TP_EQUIPOS para extraer el Id_equipo y cruzar C4M y H4M

In [0]:
# Información de la conexión a PostgreSQL
host = "srv-postgres-d4m.postgres.database.azure.com"
database = "ControlSenseDB"
user = "administrador"
password = "Protobuffers2024"

# Establecer la conexión a la base de datos
try:
    connection = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )

    # Crear un cursor para ejecutar comandos SQL
    cursor = connection.cursor()

    # Establecer la zona horaria antes de ejecutar la consulta
    time_zone_query = "SET TIME ZONE 'America/Lima';"
    cursor.execute(time_zone_query)

    # Tu consulta SQL equipos de acarreo a considerar en c4mequipos y datacamion: 61,63,64,65,68,69,71,74,103,122,125,126,198,199,201,204,205
    # equipos acarreo disponibles en telemetria h4m(2dias) : 61,65,68,126,199,201,*204*,205
    tu_query_sql = '''
    select * from ts_equipos
    --where nombre = 'FC89'
    '''
    # Ejecutar la consulta
    cursor.execute(tu_query_sql)

    # Obtener los resultados en un DataFrame de pandas
    resultados_equipos = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

    # Cerrar el cursor y la conexión
    cursor.close()
    connection.close()

    # Hacer lo que necesites con los resultados
    print(resultados_equipos.head())

except psycopg2.Error as e:
    print("Error al conectar a la base de datos PostgreSQL:", e)

In [0]:
resultados_equipos.head()

In [0]:
resultados.loc[resultados['id_equipo'] == 199]

### Datos de H4M - Api

In [0]:
import pandas as pd
from azure.storage.blob import BlobServiceClient

In [0]:
import pandas as pd
from azure.storage.blob import BlobServiceClient

# Obtener conection,de Azure DataLake,interfaz de Azure (Claves de acceso: Key1) (Debes comentar esta variable Si NO deja hacer COMMIT DEL #CODIGO EN GIT,GITHUB)
connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakemlopsd4m;AccountKey=iWT8t74/#XlqcqoR03keDVtFZPzr0PB9zDffMPaLWMUBIAjUww8uYAVkc9xRkcBtvTmUHKBvd1sB3+ASt6mGgcQ==;EndpointSuffix=core.windows.net'

# Crear el cliente del servicio Blob
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Nombre del contenedor en el Blob Storage
container_name = "raw/proyectocongestion_raw/fuentedatos_h4m/operacion_shougang/equipo_fc89"


# Lista para almacenar los DataFrames de cada archivo CSV
dfs = []

# Nombres de los archivos CSV
csv_files = ["fc89_allvar_00_7_1marzo.csv","fc89_allvar_7_13_1marzo.csv","fc89_allvar_13_19_1marzo.csv","fc89_allvar_19_2359_1marzo.csv"]

# Leer cada archivo CSV y guardarlos en DataFrames individuales
for csv_file in csv_files:
    # Descargar el archivo CSV del Blob Storage
    with open(csv_file, "wb") as my_blob:
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=csv_file)
        blob_data = blob_client.download_blob()
        blob_data.readinto(my_blob)
    
    # Leer el archivo CSV en un DataFrame de Pandas
    df = pd.read_csv(csv_file)
    
    # Agregar el DataFrame a la lista
    dfs.append(df)

# Concatenar todos los DataFrames en uno solo al final
result_df = pd.concat(dfs, ignore_index=True)

# Visualizar el DataFrame resultante
result_df.head()

In [0]:
result_df.shape